In [9]:
#!pip install pyspark

In [1]:
#spark sql imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#spark ML imports
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer, CountVectorizer, IDF
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [5]:
spark = SparkSession.builder.appName('BD_Project').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])
# the 4g is the total mun of the memory 4g * 4 executor = 16g


#print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.driver.memory', '4g'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1582732507476'),
 ('spark.executor.cores', '4'),
 ('spark.cores.max', '4'),
 ('spark.driver.port', '50496'),
 ('spark.app.name', 'Spark Updated Conf'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', 'LAPTOP-67Q35R8F.mshome.net'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [6]:
spark.sparkContext

<SparkContext master=local[*] appName=ChicagoFoodInspectionML>

## RDD

In [7]:
#store the Spark Context in a variable. Sometimes the Context is provided directly from the environment
sc = spark.sparkContext

#generate 20 natural numbers
a = range(20)

#create an RDD using Parallelize
rdd = sc.parallelize(a,3)
rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

In [8]:
#count number of elements in x
rdd.count()

20

## Load Data

In aggregate, each match's meta information and player statistics are summarized (as provided by pubg). It includes various aggregate statistics such as player kills, damage, distance walked, etc as well as metadata on the match itself such as queue size, fpp/tpp, date, etc.

In [144]:
agg_match = spark.read.csv("C:/Users/kailf/Python_code/UChicago/Big_Data/Project/agg_match_stats_0.csv", header=True)

In deaths, the files record every death that occurred within the 720k matches. That is, each row documents an event where a player has died in the match.

In [10]:
kill_match = spark.read.csv("C:/Users/kailf/Python_code/UChicago/Big_Data/Project/kill_match_stats_final_0.csv", header=True)

In [135]:
kill_match.show(3)

+---------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|killed_by|     killer_name|killer_placement|killer_position_x|killer_position_y|    map|            match_id|time|    victim_name|victim_placement|victim_position_x|victim_position_y|
+---------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|  Grenade| KrazyPortuguese|             5.0|         657725.1|         146275.2|MIRAMAR|2U4GBNA0YmnLSqvEy...| 823|KrazyPortuguese|             5.0|         657725.1|         146275.2|
|   SCAR-L|nide2Bxiaojiejie|            31.0|         93091.37|         722236.4|MIRAMAR|2U4GBNA0YmnLSqvEy...| 194|    X3evolution|            33.0|         92238.68|         723375.1|
|     S686|        Ascholes|            43.0|         366921.4|         421

In [111]:
agg_match.show(1)

+--------------------+---------+--------------------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-----------+-------------------+-------+--------------+
|                date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|player_dist_ride|player_dist_walk|player_dmg|player_kills|player_name|player_survive_time|team_id|team_placement|
+--------------------+---------+--------------------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-----------+-------------------+-------+--------------+
|2017-11-26T20:59:...|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|        2870.724|      1784.84778|       117|           1|   SnuffIes|            1106.32|      4|            18|
+--------------------+---------+--------------------+----------+----------+--------------+-----------+----------------+----------------+

In [57]:
#agg_match = agg_match.withColumn('date', F.regexp_replace('date', "+0000", ""))

In [136]:
agg_match.select('date').take(4)

[Row(date='2017-11-26T20:59:40+0000'),
 Row(date='2017-11-26T20:59:40+0000'),
 Row(date='2017-11-26T20:59:40+0000'),
 Row(date='2017-11-26T20:59:40+0000')]

In [ ]:
## EDA


In [147]:
agg_match.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- game_size: string (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: string (nullable = true)
 |-- player_assists: string (nullable = true)
 |-- player_dbno: string (nullable = true)
 |-- player_dist_ride: string (nullable = true)
 |-- player_dist_walk: string (nullable = true)
 |-- player_dmg: string (nullable = true)
 |-- player_kills: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: string (nullable = true)
 |-- team_id: string (nullable = true)
 |-- team_placement: string (nullable = true)



In [145]:
agg_match = agg_match.withColumn("date", F.unix_timestamp(agg_match["date"], "yyyy-MM-dd'T'HH:mm:ss'+0000'").cast('timestamp'))

In [138]:
agg_match = agg_match.withColumn("player_dist_ride", agg_match["player_dist_ride"].cast(IntegerType()))
agg_match = agg_match.withColumn("player_dist_walk", agg_match["player_dist_walk"].cast(FloatType()))
agg_match = agg_match.withColumn("player_dmg", agg_match["player_dmg"].cast(FloatType()))

#agg_match = agg_match.withColumn("player_dmg", agg_match["player_dmg"].cast(IntegerType()))
#agg_match = agg_match.withColumn("player_dmg", agg_match["player_dmg"].cast(IntegerType()))

In [150]:
from pyspark.sql.functions import month, year, hour, minute, second

agg_match = agg_match.withColumn('month', month(agg_match['date']))
agg_match = agg_match.withColumn('year', year(agg_match['date']))
agg_match = agg_match.withColumn('time', time(agg_match['date']))

NameError: name 'time' is not defined

In [149]:
agg_match.show(3)

+-------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+-----------+-------------------+-------+--------------+-----+----+----+
|               date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|  player_dist_ride|  player_dist_walk|player_dmg|player_kills|player_name|player_survive_time|team_id|team_placement|month|year|hour|
+-------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+-----------+-------------------+-------+--------------+-----+----+----+
|2017-11-26 20:59:40|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|          2870.724|        1784.84778|       117|           1|   SnuffIes|            1106.32|      4|            18|   11|2017|  20|
|2017-11-26 20:59:40|       37|2U4GBNA0YmnNZYkzj...|       t

Game time: more players and less waiting time / by match type


Average match time / by match type / party size


How many matches in the dataset  / by match type / party size

In [114]:
match_map=kill_match.select(['map','match_id'])

In [115]:
match_map.show(3)

+-------+--------------------+
|    map|            match_id|
+-------+--------------------+
|MIRAMAR|2U4GBNA0YmnLSqvEy...|
|MIRAMAR|2U4GBNA0YmnLSqvEy...|
|MIRAMAR|2U4GBNA0YmnLSqvEy...|
+-------+--------------------+
only showing top 3 rows



In [116]:
agg_match_joined = agg_match.join(match_map, agg_match['match_id'] == match_map['match_id'], how = 'left').drop('match_id')

In [117]:
agg_match_joined.show(3)

+--------------------+---------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-------------+-------------------+-------+--------------+-------+
|                date|game_size|match_mode|party_size|player_assists|player_dbno|player_dist_ride|player_dist_walk|player_dmg|player_kills|  player_name|player_survive_time|team_id|team_placement|    map|
+--------------------+---------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-------------+-------------------+-------+--------------+-------+
|2017-12-30T19:06:...|       94|       tpp|         1|             0|          0|               0|       23.053623|      22.0|           0|JackFFFFFFFFF|            101.117| 100010|            87|ERANGEL|
|2017-12-30T19:06:...|       94|       tpp|         1|             0|          0|               0|       23.053623|      22.0|           0|JackFFFFFFFFF|            101.117| 100010

In [104]:
agg_match_joined.groupby('map','party_size').count().show()

+-------+----------+---------+
|    map|party_size|    count|
+-------+----------+---------+
|MIRAMAR|         4|111358780|
|ERANGEL|         4|463783134|
|ERANGEL|         1|220202880|
|MIRAMAR|         2| 63433144|
|ERANGEL|         2|335063305|
|   null|         1|  1217315|
|MIRAMAR|         1| 39025182|
|   null|         4|  8230127|
|   null|         2|  5616203|
+-------+----------+---------+



Average distance walk/ride 


In [118]:
agg_match_joined.groupby('map','party_size').avg('player_dist_walk').show()

+-------+----------+---------------------+
|    map|party_size|avg(player_dist_walk)|
+-------+----------+---------------------+
|MIRAMAR|         4|    1410.742018224071|
|ERANGEL|         4|   1355.8162957032268|
|ERANGEL|         1|    1150.693641124211|
|MIRAMAR|         2|   1276.2859497094053|
|ERANGEL|         2|   1221.9183791044488|
|   null|         1|   1022.9180469875213|
|MIRAMAR|         1|   1182.2890436595835|
|   null|         4|    1299.164686919216|
|   null|         2|     1187.68092500758|
+-------+----------+---------------------+



In [119]:
agg_match_joined.groupby('map','party_size').avg('player_dist_ride').show()

+-------+----------+---------------------+
|    map|party_size|avg(player_dist_ride)|
+-------+----------+---------------------+
|MIRAMAR|         4|   1052.4923974292822|
|ERANGEL|         4|   1346.2549098260222|
|ERANGEL|         1|     909.324541313901|
|MIRAMAR|         2|    972.9549795766075|
|ERANGEL|         2|    1176.657920711431|
|   null|         1|    842.3804298805157|
|MIRAMAR|         1|    744.0219518258749|
|   null|         4|   1260.7400371829012|
|   null|         2|   1100.2779096482088|
+-------+----------+---------------------+

